In [ ]:
import docx
import jieba
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
# 加载docx文档
def load_docx(file_path):
    doc = docx.Document(file_path)
    text = []
    for para in doc.paragraphs:
        text.append(para.text)
    return "\n".join(text)

# 使用jieba进行中文分词
def jieba_tokenizer(text):
    return list(jieba.cut(text))

# 将文本转换为TF-IDF向量
def text_to_tfidf(texts):
    vectorizer = TfidfVectorizer(tokenizer=jieba_tokenizer)
    tfidf_matrix = vectorizer.fit_transform(texts).toarray()
    return tfidf_matrix, vectorizer

# 使用FAISS进行相似度检索
def search_similar_sentences(query_vector, index, k=3):
    distances, indices = index.search(query_vector, k)
    return distances, indices

In [12]:
# 示例用法
docx_path = 'test.docx'  # 替换为你的docx文件路径
keywords = ['前锋', '中场', '后卫', '守门员']  # 你要查找的多个关键词

# 加载文档内容
doc_text = load_docx(docx_path)

# 按段落分割文本
sentences = doc_text.split('\n')

# 将文本转换为TF-IDF向量
tfidf_matrix, vectorizer = text_to_tfidf(sentences)

# 构建FAISS索引
dimension = tfidf_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(tfidf_matrix, dtype=np.float32))

In [15]:
# 对每个关键词执行检索
for keyword in keywords:
    # 使用关键词生成查询向量
    query_vector = vectorizer.transform([keyword]).toarray().astype(np.float32)

    # 执行检索，查找最相似的3个句子
    distances, indices = search_similar_sentences(query_vector, index, k=3)

    # 打印查询结果
    print(f"\n查询关键词: {keyword}")
    print(f"最相似的 {len(distances[0])} 个句子:")

    for i in range(len(distances[0])):
        sentence = sentences[indices[0][i]].strip()
        # 如果句子不为空，则输出
        if sentence:
            print(f"第 {i} 名: {sentence} (距离: {distances[0][i]:.4f})")


查询关键词: 前锋
最相似的 3 个句子:
第 1 名: 前锋：刘诚宇（上海申花）、韦世豪（成都蓉城）、王子铭（北京国安）、张玉宁（北京国安）、林良铭（北京国安）、王钰栋（浙江）、拜合拉木·阿卜杜外力（深圳新鹏城）； (距离: 1.7755)
第 2 名: 以下为国足世预赛18强赛大名单： (距离: 2.0000)

查询关键词: 中场
最相似的 3 个句子:
第 1 名: 为备战18强赛，国足主教练伊万科维奇此前征召了32名球员，从3月3日起在阿联酋迪拜组织集训。随后球队又补招了山东泰山中场李源一入队。效力于北京国安的塞尔吉尼奥在3月14日正式获得代表中国队出场的资格。国足在15日的一场封闭热身赛中3:1击败科威特队，塞尔吉尼奥替补出场。 (距离: 1.8139)
第 2 名: 中场：徐皓阳（上海申花）、汪海健（上海申花）、曹永竞（北京国安）、塞尔吉尼奥（北京国安）、李源一（山东泰山）、谢文能（山东泰山）、程进（浙江）、王上源（河南）； (距离: 1.8211)

查询关键词: 后卫
最相似的 3 个句子:
第 1 名: 后卫：蒋光太（上海海港）、魏震（上海海港）、王振澳（上海海港）、杨泽翔（上海申花）、蒋圣龙（上海申花）、韩鹏飞（成都蓉城）、胡荷韬（成都蓉城）、李磊（北京国安）、高准翼（山东泰山）； (距离: 1.8265)
第 2 名: 新华社北京3月16日电 16日，中国男足国家队公布了参加2026美加墨世界杯亚洲区预选赛18强赛第七和第八轮比赛的27人大名单，归化球员塞尔吉尼奥首次入选国家队，两位2006年出生的小将王钰栋、刘诚宇名列其中。 (距离: 2.0000)

查询关键词: 守门员
最相似的 3 个句子:
第 1 名: 守门员：颜骏凌（上海海港）、刘殿座（成都蓉城）、王大雷（山东泰山）。 (距离: 1.5350)
第 2 名: 为备战18强赛，国足主教练伊万科维奇此前征召了32名球员，从3月3日起在阿联酋迪拜组织集训。随后球队又补招了山东泰山中场李源一入队。效力于北京国安的塞尔吉尼奥在3月14日正式获得代表中国队出场的资格。国足在15日的一场封闭热身赛中3:1击败科威特队，塞尔吉尼奥替补出场。 (距离: 2.0000)
